In [1]:
import io
import opvious
import pandas as pd

client = opvious.Client.from_environment()

In [70]:
demand_df = pd.read_csv(io.StringIO("""
location,tier,product,size,demand
Boston,T1,hoodie,M,50
Boston,T1,shirt,L,30
Boston,T2,shirt,L,25
Boston,T1,shirt,XL,20
Seattle,T1,hoodie,M,100
Seattle,T1,hoodie,L,75
Seattle,T2,hoodie,L,50
Seattle,T1,hoodie,XL,50
Seattle,T1,shirt,L,10
""")).set_index(["location", "product", "size", "tier"])
demand_df

demand
location product size tier        
Boston   hoodie  M    T1        50
         shirt   L    T1        30
                      T2        25
                 XL   T1        20
Seattle  hoodie  M    T1       100
                 L    T1        75
                      T2        50
                 XL   T1        50
         shirt   L    T1        10

In [71]:
supply_df = pd.read_csv(io.StringIO("""
product,size,supply
hoodie,M,100
hoodie,L,50
shirt,L,50
shirt,XL,10
""")).set_index(["product", "size"])
supply_df

supply
product size        
hoodie  M        100
        L         50
shirt   L         50
        XL        10

In [76]:
product_df = pd.read_csv(io.StringIO("""
product,min_allocation,diversity
hoodie,100,2
shirt,10,2
""")).set_index(["product"])
product_df

,min_allocation,diversity
product,,
hoodie,100,2
shirt,10,2


In [77]:
inputs = await client.assemble_inputs(
    formulation_name="product-allocation",
    parameters={
        "demand": demand_df["demand"],
        "demandValue": {"T1": 1, "T2": 0.8},
        "minAllocation": product_df["min_allocation"],
        "productDiversity": product_df["diversity"],
        "supply": supply_df["supply"],
        "maxTotalAllocation": 500
    },
    infer_dimensions=True
)

In [78]:
attempt = await client.start_attempt(inputs)
outcome = await client.wait_for_outcome(attempt)

Tracking attempt... [url=https://hub.beta.opvious.io/attempts/b3cfbd6f-6222-4715-a112-a12233664c6e]
Attempt is running... [elapsed=227 milliseconds]
Attempt is optimal. [elapsed=619 milliseconds, objective=208, gap=0.0%]


In [82]:
await client.fetch_variable(attempt, label="allocation")

value
location product size demandTier       
Boston   shirt   L    T1             30
                      T2             10
                 XL   T1             10
Seattle  hoodie  L    T1             50
                 M    T1            100
         shirt   L    T1             10